# Imports

In [1]:
import pandas as pd
import numpy as np
import json
import requests

**installing spotipy**

In [2]:
!pip install spotipy

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Setting the credencials

In [13]:
client_id = '2753defd10e2496f80b0d2348c2b5481'
client_secret = '34a98fb5da1b4c7698f93c077db6b127'

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))


# Prepare data

In [14]:
# Playlist id
pl_id = 'spotify:playlist:37i9dQZF1DX18jTM2l2fJY'

# Getting the tracks in the playlist
response = sp.playlist(pl_id)

# How many tracks in the playlist
len(response['tracks']['items'])

50

## Getting features ready to turn into dataframe

In [15]:
name = [response['tracks']['items'][i]['track']['name'] for i in range(len(response['tracks']['items']))]

In [16]:
artist = [response['tracks']['items'][i]['track']['artists'][0]['name'] for i in range(len(response['tracks']['items']))]

In [17]:
duration_ms = [response['tracks']['items'][i]['track']['duration_ms'] for i in range(len(response['tracks']['items']))]

In [18]:
popularity = [response['tracks']['items'][i]['track']['popularity'] for i in range(len(response['tracks']['items']))]

In [19]:
Id = [response['tracks']['items'][i]['track']['id'] for i in range(len(response['tracks']['items']))]

In [20]:
href = [response['tracks']['items'][i]['track']['href'] for i in range(len(response['tracks']['items']))]

In [21]:
release_date = [response['tracks']['items'][i]['track']['album']['release_date'] for i in range(len(response['tracks']['items']))]

### 1st Dataframe ready

In [22]:
data = {}

# Setting the features
data['name'] = name
data['artist'] = artist
data['duration_ms'] = duration_ms
data['popularity'] = popularity
data['id'] = Id
data['href'] = href
data['release_date'] = release_date

# turning into dataframe
df = pd.DataFrame(data)

#turninge the realise date into datetime
df['release_date'] = pd.to_datetime(df['release_date'])

### 2nd Datarame ready

In [23]:
# List of features set
tracks = []

# For each track in the 1st dataframe 
for track_id in df.loc[:,'id']:
    track = f'spotify:track:{track_id}' # Get the track features
    audio_features = sp.audio_features(track) # Save in audio features variable
    tracks.append(audio_features[0]) # Append to the tracks list

In [24]:
# New dataframe with all the tracks in the first dataframe
df_features =  pd.DataFrame(tracks)
df_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,0.761,0.525,11,-6.900,1,0.0944,0.4400,0.000007,0.0921,0.531,80.870,audio_features,02MWAaffLxlfxAUY7c5dvx,spotify:track:02MWAaffLxlfxAUY7c5dvx,https://api.spotify.com/v1/tracks/02MWAaffLxlf...,https://api.spotify.com/v1/audio-analysis/02MW...,238805,4
2,0.591,0.764,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,audio_features,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,141806,4
3,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
4,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,audio_features,1IHWl5LamUGEuP4ozKQSXZ,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4


In [25]:
# Inner merging between the 2 dataframe on id
data = df.merge(df_features, on='id')
data.head()

,name,artist,duration_ms_x,popularity,id,href,release_date,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms_y,time_signature
0,As It Was,Harry Styles,167303,90,4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,2022-03-31,0.520,0.731,6,...,0.001010,0.3110,0.662,173.930,audio_features,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,Heat Waves,Glass Animals,238805,84,02MWAaffLxlfxAUY7c5dvx,https://api.spotify.com/v1/tracks/02MWAaffLxlf...,2020-08-06,0.761,0.525,11,...,0.000007,0.0921,0.531,80.870,audio_features,spotify:track:02MWAaffLxlfxAUY7c5dvx,https://api.spotify.com/v1/tracks/02MWAaffLxlf...,https://api.spotify.com/v1/audio-analysis/02MW...,238805,4
2,STAY (with Justin Bieber),The Kid LAROI,141805,85,5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,2021-07-09,0.591,0.764,1,...,0.000000,0.1030,0.478,169.928,audio_features,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,141806,4
3,Me Porto Bonito,Bad Bunny,178567,90,6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,2022-05-06,0.911,0.712,1,...,0.000027,0.0933,0.425,92.005,audio_features,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
4,Tití Me Preguntó,Bad Bunny,243716,89,1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,2022-05-06,0.650,0.715,5,...,0.000291,0.1260,0.187,106.672,audio_features,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4


In [26]:
data.shape

(50, 24)